# VGG 16 (2014)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

D:\Users\Евгений Артёменко\PycharmProjects\Projects\Machine learning\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Этап 1: Подготовка данных

In [4]:
class CatDogsDataset(torch.utils.data.Dataset):
    def __init__(self, path_dir1:str, path_dir2:str):
        super().__init__()
        
        self.path_dir1 = path_dir1
        self.path_dir2 = path_dir2
        
        self.dir1_list = sorted(os.listdir(path_dir1))
        self.dir2_list = sorted(os.listdir(path_dir2))
        
    def __len__(self):
        return len(self.dir1_list) + len(self.dir2_list)
    
    def __getitem__(self, idx):
        
        if idx < len(self.dir1_list):
            class_id = 0
            img_path = os.path.join(self.path_dir1, self.dir1_list[idx])
        else:
            class_id = 1
            idx -= len(self.dir1_list)
            img_path = os.path.join(self.path_dir2, self.dir2_list[idx])
        
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)
        img = img/255.0
        
        img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)
        img = img.transpose((2, 0, 1))
        
        t_img = torch.from_numpy(img)
        t_class_id = torch.tensor(class_id)
        
        return {'img': t_img, 'label': t_class_id}

In [5]:
train_dogs_path = 'training_set\dogs'
train_cats_path = 'training_set\cats'

test_dogs_path = 'test_set\dogs'
test_cats_path = 'test_set\cats'

train_ds = CatDogsDataset(train_dogs_path, train_cats_path)
test_ds = CatDogsDataset(test_dogs_path, test_cats_path)

## Этап 2: DataLoader

In [6]:
batch_size = 4

train_loader = torch.utils.data.DataLoader(
    train_ds, shuffle=True, 
    batch_size=batch_size, num_workers=1, drop_last=True
)
test_loader = torch.utils.data.DataLoader(
    test_ds, shuffle=False,
    batch_size=batch_size, num_workers=1, drop_last=False
)

## Этап 3: Архитектура нейроннной сети

In [7]:
class VGG16(nn.Module):
    def __init__(self, out_nc):
        super().__init__()
        
        self.activation = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(2, 2)
        
        self.conv0 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        
        self.conv7 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv8 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv9 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        
        self.conv10 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv11 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv12 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        
        self.flat = nn.Flatten()
        
        self.linear1 = nn.Linear(7*7*512, 4096)
        self.linear2 = nn.Linear(7*7*512, 4096)
        self.linear3 = nn.Linear(7*7*512, out_nc)

        
    def forward(self, X):
        out = self.conv0(X)
        out = self.activation(out)
        out = self.conv1(out)
        out = self.activation(out)
        
        out = self.maxpool(out)
        
        out = self.conv2(out)
        out = self.activation(out)
        out = self.conv3(out)
        out = self.activation(out)
        
        out = self.maxpool(out)
        
        out = self.conv4(out)
        out = self.activation(out)
        out = self.conv5(out)
        out = self.activation(out)
        out = self.conv6(out)
        out = self.activation(out)
        
        out = self.maxpool(out)
        
        out = self.conv7(out)
        out = self.activation(out)
        out = self.conv8(out)
        out = self.activation(out)
        out = self.conv9(out)
        out = self.activation(out)
        
        out = self.maxpool(out)
        
        out = self.conv10(out)
        out = self.activation(out)
        out = self.conv11(out)
        out = self.activation(out)
        out = self.conv12(out)
        out = self.activation(out)
        
        out = self.maxpool(out)
        
        out = self.flat(out)
        
        out = self.linear1(out)
        out = self.activation(out)
        out = self.linear1(out)
        out = self.activation(out)
        out = self.linear1(out)
        out = self.activation(out)
        
        return out

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def accuracy(pred, label):
    answer = F.softmax(pred.detach()).numpy().argmax(1) == label.numpy().argmax(1)
    return answer.mean()

In [9]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
model = VGG16(1)
# model = model.to(device)
count_parameters(model)

220268865

In [11]:
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, betas=(0.9, 0.999))

## Этап 4: Обучение нейронной сети

In [ ]:
epochs = 7

for epoch in range(epochs):
    loss_val = 0
    acc_val = 0
    for sample in (pbar := tqdm(train_loader)):
        img, label = sample['img'], sample['label']
#         img = img.to(device)
#         label = label.float().to(device)
        optimizer.zero_grad()
        
#         label = F.one_hot(label, 2).float()
        pred = model(img)
        
        loss = loss_func(pred,label)
        loss.backward()
        loss_val += loss.item()
        
        optimizer.step()
        
        acc_val += accuracy(pred, label)
        
    pbar.set_description(f'loss: {loss.item()} \ accuracy: {accuracy(pred, label)}')
    print(loss_val/len(train_loader))
    print(acc_val/len(train_loader))